In [1]:
import ollama
from ollama import chat
from ollama import ChatResponse

import pandas as pd
import numpy as np

import json
import re
import requests
import time

In [2]:
# # response: ChatResponse = chat(model='qwq:32b', messages=[
# #   {
# #     'role': 'user',
# #     'content': 'Why is the sky blue?',
# #   },
# # ])
# # print(response['message']['content'])

# from ollama import Client
# client = Client(
#   host='http://localhost:11434',
#   headers = {
#         "Content-Type": "application/json"
#     }
# )

In [5]:
df = pd.read_pickle("./dataset/processed.pkl")  
df

columns = [
    'question',
    'ground_truth',
    'student_incorrect_solution',
    'student_profile',
    'conversation',
    'state',
    'action_text',
    'done'
]

new_df = df[columns]

state_df = new_df.copy()
state_df['full_state'] = None

for i in range(0, len(new_df)):
  if 'START' in str(new_df.iloc[i]['state']):
    state_df.at[i, 'full_state'] = ['', 'START']
    continue

  prev_action_text = new_df.iloc[i-1]['action_text']
  curr_state = new_df.iloc[i]['state']

  state_df.at[i, 'full_state'] = [prev_action_text, curr_state]

state_df

state_df['conversation']
curr_pos = 1
if len(state_df) > 0:
  state_df.loc[0, 'convo_turn'] = curr_pos

for i in range(1, len(state_df)):
    # Check if a new conversation starts (0 follows a 1)
    if state_df.loc[i-1, 'done'] == 1 and state_df.loc[i, 'done'] == 0:
        # Reset position counter for new conversation
        curr_pos = 1
    else:
        # Increment position within the same conversation
        curr_pos += 1

    # Assign the current position within conversation
    state_df.loc[i, 'convo_turn'] = curr_pos * 1

state_df['convo_turn'] = state_df['convo_turn'].astype(int)
state_df['old_state'] = state_df['state']
state_df['state'] = state_df['full_state']
state_df

,question,ground_truth,student_incorrect_solution,student_profile,conversation,state,action_text,done,full_state,convo_turn,old_state
0,Nancy is filling an aquarium for her fish. She...,First calculate the volume of the aquarium by ...,The aquarium has a volume of 4 x 6 x 3 = 72 cu...,Steven is a 7th grade student. He has difficul...,"Teacher: (probing)Steven, If you had 4 of some...","[, START]","Teacher: (probing)Steven, If you had 4 of some...",0,"[, START]",1,START
1,Nancy is filling an aquarium for her fish. She...,First calculate the volume of the aquarium by ...,The aquarium has a volume of 4 x 6 x 3 = 72 cu...,Steven is a 7th grade student. He has difficul...,"Teacher: (probing)Steven, If you had 4 of some...","[Teacher: (probing)Steven, If you had 4 of som...",Teacher: (probing)So if Nancy triples the 18 c...,0,"[Teacher: (probing)Steven, If you had 4 of som...",2,Steven: I would have 12 of something.
2,Nancy is filling an aquarium for her fish. She...,First calculate the volume of the aquarium by ...,The aquarium has a volume of 4 x 6 x 3 = 72 cu...,Steven is a 7th grade student. He has difficul...,"Teacher: (probing)Steven, If you had 4 of some...",[Teacher: (probing)So if Nancy triples the 18 ...,Teacher: (generic)Exactly correct!,1,[Teacher: (probing)So if Nancy triples the 18 ...,3,Steven: She would have 54 cubic feet of water.
3,John is very unfit and decides to work up to d...,He needs to do 15*3=45 progressions\nThat will...,"To get to 15 reps, John will take 15 - 1 = 14 ...",Stephanie is a 7th grade student. She has diff...,"Teacher: (probing)Stephanie, How many days wil...","[, START]","Teacher: (probing)Stephanie, How many days wil...",0,"[, START]",1,START
4,John is very unfit and decides to work up to d...,He needs to do 15*3=45 progressions\nThat will...,"To get to 15 reps, John will take 15 - 1 = 14 ...",Stephanie is a 7th grade student. She has diff...,"Teacher: (probing)Stephanie, How many days wil...","[Teacher: (probing)Stephanie, How many days wi...",Teacher: (probing)So how many days will it tak...,0,"[Teacher: (probing)Stephanie, How many days wi...",2,"Student: It will take one day to do one step, ..."
...,...,...,...,...,...,...,...,...,...,...,...
20149,Andy is checking to see if all the cars in the...,First find how many cars have valid tickets: 7...,"75% of the cars have valid tickets, so 100%-75...",Stephanie is a 7th grade student. She struggle...,"Teacher: (generic)Hi, could you please walk me...","[Teacher: (generic)Hi, could you please walk m...",Teacher: (focus)That doesn't seem to be correc...,0,"[Teacher: (generic)Hi, could you please walk m...",2,"Student: Sure, I said that 75% of the cars hav..."
20150,Andy is checking to see if all the cars in the...,First find how many cars have valid tickets: 7...,"75% of the cars have valid tickets, so 100%-75...",Stephanie is a 7th grade student. She struggle...,"Teacher: (generic)Hi, could you please walk me...",[Teacher: (focus)That doesn't seem to be corre...,Teacher: (focus)Great. The question now says t...,0,[Teacher: (focus)That doesn't seem to be corre...,3,Student: Sure. I said that 75% of the cars hav...
20151,Andy is checking to see if all the cars in the...,First find how many cars have valid tickets: 7...,"75% of the cars have valid tickets, so 100%-75...",Stephanie is a 7th grade student. She struggle...,"Teacher: (generic)Hi, could you please walk me...",[Teacher: (focus)Great. The question now says ...,Teacher: (focus)That's wonderful. So now what ...,0,[Teacher: (focus)Great. The question now says ...,4,"Student: Yes, 1/5th of 225 cars is (1/5)*225 =..."
20152,Andy is checking to see if all the cars in the...,First find how many cars have valid tickets: 7...,"75% of the cars have valid tickets, so 100%-75...",Stephanie is a 7th grade student. She struggle...,"Teacher: (generic)Hi, could you please walk me...",[Teacher: (focus)That's wonderful. So now what...,Teacher: (generic)There's your answer. I hope ...,0,[Teacher: (focus)T

# Prompt

In [9]:
# for i, row in state_df.iterrows():
#     if i == 5:
#         break
#     # Gather your fields
#     question = row.get("question", "")
#     correct_solution = row.get("ground_truth", "")
#     student_profile = row.get("student_profile", "")
#     current_conversation_state = row.get("state", "")
#     conversation_turn = row.get("convo_turn", "")
#     action_text = row.get("action_text", "")
#     prompt = f"""
# You are analyzing math tutoring sessions to extract features for an AI tutor.

# CONTEXT:
# - Question: {question}
# - Correct solution: {correct_solution}
# - Student profile: {student_profile}
# - Current conversation: {current_conversation_state}
# - Conversation turn: {conversation_turn}
# - Next action text: {action_text}

# TASK:
# Extract the following features from this tutoring interaction:

# 1. MISCONCEPTION TYPE - This classification MUST be determined SOLELY from the student profile, not from the conversation. It represents the student's core misunderstanding and should be CONSISTENT across all interactions with this same student. Choose ONE of these types based only on the student profile description:
# - operation_error: Student applies the wrong operation (e.g., adding instead of multiplying)
# - number_fact_error: Student makes errors in basic number facts (e.g., multiplication tables)
# - variable_meaning_error: Student misunderstands what variables represent
# - conceptual_error: Student has fundamental misconception about a math concept
# - calculation_error: Student makes arithmetic errors during calculation
# - problem_interpretation_error: Student misunderstands what the problem is asking

# 2. LISTEN TO FEEDBACK:
#    - 1 if the student has incorporated the teacher's feedback in the current conversation state
#    - 0 if not

# 3. PROBLEM PROGRESS - Numerical estimate (0-100) of how close, given the student's current conversation state, the student is to the correct solution. For example:
#   - 0: No progress/completely wrong approach
#   - 25: Beginning to understand but major errors remain
#   - 50: Halfway to solution with some key insights
#   - 75: Minor errors but mostly correct approach
#   - 100: Complete and correct solution
#   (You may use any value between 0-100, not just the examples above)

# 4. PROBLEM CONCEPT:
#    The specific math domain/concept being tested. Based on the question, choose the one most relevant concept: "basic_arithmetic", "fractions_decimals_and_percents", "ratios_proportions_and_rates", "measurement_and_geometry", "counting_and_combinatorics", "money_and_finance", or "algebraic_relationships".

# 5. CONVERSATION TURN:
#    The number of exchanges so far.

# 6. CORRECT SOLUTION:
#    - 0 if the student's final answer is incorrect
#    - 1 if correct

# 7. PROBLEM DIFFICULTY (1-3):
#    Rate how difficult the math question is on a scale from 1 to 5, where:
#    - 1 = Very easy, basic concepts, straightforward approach
#    - 2 = Moderate, requires some thought, multiple steps
#    - 3 = Challenging, complex approach, requires deeper understanding

# 8. NEXT ACTION HINT STRENGTH (0-4):
#    How strong is the teacher's hint? (0=very minimal hint, 4=essentially giving away the entire solution)

# 9. NEXT ACTION CATEGORY:
#    - Focus
#    - Probing
#    - Telling
#    - Generic

# 10. NEXT ACTION SUBCATEGORY:
#     Choose **exactly one** of the following, depending on the category:

#     **Focus**:
#     - "Seek Next Step"
#     - "Confirm Calculation"
#     - "Re-direct to Sub-Problem"
#     - "Highlight Missing Info"

#     **Probing**:
#     - "Ask for Explanation"
#     - "Seek Self-Correction"
#     - "Hypothetical Variation"
#     - "Check Understanding/Concept"
#     - "Encourage Comparison"

#     **Telling**:
#     - "Partial Reveal (Strategy)"
#     - "Full Reveal (Answer)"
#     - "Corrective Explanation"

#     **Generic**:
#     - "Greeting/Farewell"
#     - "Acknowledgment/Praise"
#     - "Summarize Progress"
#     - "General Inquiry/Filler"

# FORMAT YOUR RESPONSE AS JSON:
# {{
#   "state": {{
#     "misconception_type": "one_of_the_six_types",
#     "listen_to_feedback": 0 or 1,
#     "problem_progress": integer_between_0_and_100,
#     "problem_concept": "specific_math_concept",
#     "conversation_turn": integer,
#     "correct_solution": 0 or 1,
#     "problem_difficulty": integer_between_1_and_3
#   }},
#   "action": {{
#     "next_action_hint_strength": integer_between_0_and_4,
#     "next_action_category": "Focus/Probing/Telling/Generic",
#     "next_action_subcategory": "pick_the_exact_one_subcategory_name"
#   }}
# }}

# IMPORTANT:
# - Return ONLY the JSON object with no additional explanation
# - All numeric fields must be valid integers
# """
#     print(i, prompt)

0 
You are analyzing math tutoring sessions to extract features for an AI tutor.

CONTEXT:
- Question: Nancy is filling an aquarium for her fish. She fills it halfway and goes to answer the door. While she's gone, her cat knocks the aquarium over and spills half the water in it. Then Nancy comes back and triples the amount of water in the aquarium. If the aquarium is 4 feet long, 6 feet wide, and 3 feet high, how many cubic feet of water are in the aquarium?
- Correct solution: First calculate the volume of the aquarium by multiplying its length, width and height: 4 ft * 6 ft * 3 ft = 72 cubic ft
Then figure out what proportion of the aquarium is full after the cat knocks it over: 1/2 * 1/2 = 1/4
Then figure out what proportion of the aquarium is full after Nancy refills it: 3 * 1/4 = 3/4
Now multiply the proportion of the aquarium that's full by the aquarium's volume to find out how much water is in it: 72 cubic ft * 3/4 = 54 cubic ft
 54
- Student profile: Steven is a 7th grade stude

In [3]:
state_df

,question,ground_truth,student_incorrect_solution,student_profile,conversation,state,action_text,done,full_state,convo_turn,old_state
0,Nancy is filling an aquarium for her fish. She...,First calculate the volume of the aquarium by ...,The aquarium has a volume of 4 x 6 x 3 = 72 cu...,Steven is a 7th grade student. He has difficul...,"Teacher: (probing)Steven, If you had 4 of some...","[, START]","Teacher: (probing)Steven, If you had 4 of some...",0,"[, START]",1,START
1,Nancy is filling an aquarium for her fish. She...,First calculate the volume of the aquarium by ...,The aquarium has a volume of 4 x 6 x 3 = 72 cu...,Steven is a 7th grade student. He has difficul...,"Teacher: (probing)Steven, If you had 4 of some...","[Teacher: (probing)Steven, If you had 4 of som...",Teacher: (probing)So if Nancy triples the 18 c...,0,"[Teacher: (probing)Steven, If you had 4 of som...",2,Steven: I would have 12 of something.
2,Nancy is filling an aquarium for her fish. She...,First calculate the volume of the aquarium by ...,The aquarium has a volume of 4 x 6 x 3 = 72 cu...,Steven is a 7th grade student. He has difficul...,"Teacher: (probing)Steven, If you had 4 of some...",[Teacher: (probing)So if Nancy triples the 18 ...,Teacher: (generic)Exactly correct!,1,[Teacher: (probing)So if Nancy triples the 18 ...,3,Steven: She would have 54 cubic feet of water.
3,John is very unfit and decides to work up to d...,He needs to do 15*3=45 progressions\nThat will...,"To get to 15 reps, John will take 15 - 1 = 14 ...",Stephanie is a 7th grade student. She has diff...,"Teacher: (probing)Stephanie, How many days wil...","[, START]","Teacher: (probing)Stephanie, How many days wil...",0,"[, START]",1,START
4,John is very unfit and decides to work up to d...,He needs to do 15*3=45 progressions\nThat will...,"To get to 15 reps, John will take 15 - 1 = 14 ...",Stephanie is a 7th grade student. She has diff...,"Teacher: (probing)Stephanie, How many days wil...","[Teacher: (probing)Stephanie, How many days wi...",Teacher: (probing)So how many days will it tak...,0,"[Teacher: (probing)Stephanie, How many days wi...",2,"Student: It will take one day to do one step, ..."
...,...,...,...,...,...,...,...,...,...,...,...
20149,Andy is checking to see if all the cars in the...,First find how many cars have valid tickets: 7...,"75% of the cars have valid tickets, so 100%-75...",Stephanie is a 7th grade student. She struggle...,"Teacher: (generic)Hi, could you please walk me...","[Teacher: (generic)Hi, could you please walk m...",Teacher: (focus)That doesn't seem to be correc...,0,"[Teacher: (generic)Hi, could you please walk m...",2,"Student: Sure, I said that 75% of the cars hav..."
20150,Andy is checking to see if all the cars in the...,First find how many cars have valid tickets: 7...,"75% of the cars have valid tickets, so 100%-75...",Stephanie is a 7th grade student. She struggle...,"Teacher: (generic)Hi, could you please walk me...",[Teacher: (focus)That doesn't seem to be corre...,Teacher: (focus)Great. The question now says t...,0,[Teacher: (focus)That doesn't seem to be corre...,3,Student: Sure. I said that 75% of the cars hav...
20151,Andy is checking to see if all the cars in the...,First find how many cars have valid tickets: 7...,"75% of the cars have valid tickets, so 100%-75...",Stephanie is a 7th grade student. She struggle...,"Teacher: (generic)Hi, could you please walk me...",[Teacher: (focus)Great. The question now says ...,Teacher: (focus)That's wonderful. So now what ...,0,[Teacher: (focus)Great. The question now says ...,4,"Student: Yes, 1/5th of 225 cars is (1/5)*225 =..."
20152,Andy is checking to see if all the cars in the...,First find how many cars have valid tickets: 7...,"75% of the cars have valid tickets, so 100%-75...",Stephanie is a 7th grade student. She struggle...,"Teacher: (generic)Hi, could you please walk me...",[Teacher: (focus)That's wonderful. So now what...,Teacher: (generic)There's your answer. I hope ...,0,[Teacher: (focus)T

In [13]:
state_df["conversation"][0]

'Teacher: (probing)Steven, If you had 4 of something and tripled that amount, how much would you have?|EOM|Steven: I would have 12 of something.|EOM|Teacher: (probing)So if Nancy triples the 18 cubic feet of water, how much would she have?|EOM|Steven: She would have 54 cubic feet of water.|EOM|Teacher: (generic)Exactly correct!'

In [6]:
from ollama import Client
client = Client(
  host='http://localhost:11434',
  headers = {
        "Content-Type": "application/json"
    }
)


def extract_json(text):
    """
    Extract JSON object from text that might contain additional content,
    then validate whether it fits the expected schema.

    Returns:
        dict or None: Extracted JSON object if valid, otherwise None
    """
    print(f"Raw text (last 200 chars): {text[-200:] if len(text) > 200 else text}")

    # 1) Direct JSON loading attempt
    try:
        data = json.loads(text)
        if validate_schema(data):
            return data
    except json.JSONDecodeError:
        pass

    # 2) Regex pattern (brute-force) for JSON blocks
    json_pattern = r'\s*(\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\})\s*'
    matches = re.findall(json_pattern, text, re.DOTALL)
    if matches:
        # Try from the last match backward (often the last is the full JSON)
        for potential_json in reversed(matches):
            try:
                data = json.loads(potential_json)
                if validate_schema(data):
                    return data
            except json.JSONDecodeError:
                continue

    # 3) Aggressive brace matching from the last '{' found
    try:
        start_idx = text.rfind('{')
        if start_idx != -1:
            brace_count = 1
            i = start_idx + 1
            while i < len(text) and brace_count > 0:
                if text[i] == '{':
                    brace_count += 1
                elif text[i] == '}':
                    brace_count -= 1
                i += 1

            if brace_count == 0:
                json_str = text[start_idx:i]
                data = json.loads(json_str)
                if validate_schema(data):
                    return data
    except Exception as e:
        print(f"Error in aggressive JSON extraction: {str(e)}")

    # 4) Fallback pattern with your required fields (if you want to be very specific)
    # Adjusted to include problem_difficulty
    pattern = (
        r'\{\s*"state":\s*\{[^}]*\},\s*"action":\s*\{[^}]*\}\s*\}'
    )
    matches = re.findall(pattern, text, re.DOTALL)
    if matches:
        for potential_json in matches:
            try:
                data = json.loads(potential_json.strip())
                if validate_schema(data):
                    return data
            except json.JSONDecodeError:
                continue

    print("All extraction methods failed or schema validation failed. Returning None.")
    return None


def validate_schema(data):
    """
    Validate the JSON structure to ensure it contains the right keys and
    that numeric fields are truly numeric and in the right range, etc.
    """
    # Top-level keys we expect
    if not isinstance(data, dict):
        return False

    if "state" not in data or "action" not in data:
        return False

    # Validate 'state' keys
    state = data["state"]
    required_state_keys = [
        "misconception_type", "listen_to_feedback", "problem_progress",
        "problem_concept", "conversation_turn", "correct_solution",
        "problem_difficulty"
    ]
    for key in required_state_keys:
        if key not in state:
            return False

    # Check numeric fields
    if not isinstance(state["listen_to_feedback"], int):
        return False
    if state["listen_to_feedback"] not in (0, 1):
        return False

    if not isinstance(state["problem_progress"], int):
        return False
    if not (0 <= state["problem_progress"] <= 100):
        return False

    if not isinstance(state["conversation_turn"], int):
        return False

    if not isinstance(state["correct_solution"], int):
        return False
    if state["correct_solution"] not in (0, 1):
        return False

    # Check problem_difficulty
    if not isinstance(state["problem_difficulty"], int):
        return False
    if not (1 <= state["problem_difficulty"] <= 5):
        return False

    # Validate 'action' keys
    action = data["action"]
    required_action_keys = [
        "next_action_hint_strength", "next_action_category", "next_action_subcategory"
    ]
    for key in required_action_keys:
        if key not in action:
            return False

    # Check action numeric fields
    if not isinstance(action["next_action_hint_strength"], int):
        return False
    # Change this line in validate_schema()
    if not (0 <= action["next_action_hint_strength"] <= 4):
        return False

    # Check action category
    valid_action_categories = ["Focus", "Probing", "Telling", "Generic"]
    if action["next_action_category"] not in valid_action_categories:
        return False

    # next_action_subcategory: up to you how strictly to validate
    # For example, if Focus: could be one of [Seek Next Step, Confirm Calculation, etc.]
    # We'll do minimal checking here
    if not isinstance(action["next_action_subcategory"], str):
        return False

    return True


# Example script usage:
processed = 0
successful = 0
failed = 0

for i, row in state_df.iterrows():
    # Gather your fields
    question = row.get("question", "")
    correct_solution = row.get("ground_truth", "")
    student_profile = row.get("student_profile", "")
    current_conversation_state = row.get("state", "")
    conversation_turn = row.get("convo_turn", "")
    action_text = row.get("action_text", "")

    # Updated prompt with problem_difficulty added
    if i == 0:
        prompt = f"""
You are analyzing math tutoring sessions to extract features for an AI tutor.

CONTEXT:
- Question: {question}
- Correct solution: {correct_solution}
- Student profile: {student_profile}
- Current conversation: {current_conversation_state}
- Conversation turn: {conversation_turn}
- Next action text: {action_text}

TASK:
Extract the following features from this tutoring interaction:

1. MISCONCEPTION TYPE - This classification MUST be determined SOLELY from the student profile, not from the conversation. It represents the student's core misunderstanding and should be CONSISTENT across all interactions with this same student. Choose ONE of these types based only on the student profile description:
- operation_error: Student applies the wrong operation (e.g., adding instead of multiplying)
- number_fact_error: Student makes errors in basic number facts (e.g., multiplication tables)
- variable_meaning_error: Student misunderstands what variables represent
- conceptual_error: Student has fundamental misconception about a math concept
- calculation_error: Student makes arithmetic errors during calculation
- problem_interpretation_error: Student misunderstands what the problem is asking

2. LISTEN TO FEEDBACK:
   - 1 if the student has incorporated the teacher's feedback in the current conversation state
   - 0 if not

3. PROBLEM PROGRESS - Numerical estimate (0-100) of how close, given the student's current conversation state, the student is to the correct solution. For example:
  - 0: No progress/completely wrong approach
  - 25: Beginning to understand but major errors remain
  - 50: Halfway to solution with some key insights
  - 75: Minor errors but mostly correct approach
  - 100: Complete and correct solution
  (You may use any value between 0-100, not just the examples above)

4. PROBLEM CONCEPT:
   The specific math domain/concept being tested. Based on the question, choose the one most relevant concept: "basic_arithmetic", "fractions_decimals_and_percents", "ratios_proportions_and_rates", "measurement_and_geometry", "counting_and_combinatorics", "money_and_finance", or "algebraic_relationships".

5. CONVERSATION TURN:
   The number of exchanges so far.

6. CORRECT SOLUTION:
   - 0 if the student's final answer is incorrect
   - 1 if correct

7. PROBLEM DIFFICULTY (1-3):
   Rate how difficult the math question is on a scale from 1 to 5, where:
   - 1 = Very easy, basic concepts, straightforward approach
   - 2 = Moderate, requires some thought, multiple steps
   - 3 = Challenging, complex approach, requires deeper understanding

8. NEXT ACTION HINT STRENGTH (0-4):
   How strong is the teacher's hint? (0=very minimal hint, 4=essentially giving away the entire solution)

9. NEXT ACTION CATEGORY:
   - Focus
   - Probing
   - Telling
   - Generic

10. NEXT ACTION SUBCATEGORY:
    Choose **exactly one** of the following, depending on the category:

    **Focus**:
    - "Seek Next Step"
    - "Confirm Calculation"
    - "Re-direct to Sub-Problem"
    - "Highlight Missing Info"

    **Probing**:
    - "Ask for Explanation"
    - "Seek Self-Correction"
    - "Hypothetical Variation"
    - "Check Understanding/Concept"
    - "Encourage Comparison"

    **Telling**:
    - "Partial Reveal (Strategy)"
    - "Full Reveal (Answer)"
    - "Corrective Explanation"

    **Generic**:
    - "Greeting/Farewell"
    - "Acknowledgment/Praise"
    - "Summarize Progress"
    - "General Inquiry/Filler"

FORMAT YOUR RESPONSE AS JSON:
{{
  "state": {{
    "misconception_type": "one_of_the_six_types",
    "listen_to_feedback": 0 or 1,
    "problem_progress": integer_between_0_and_100,
    "problem_concept": "specific_math_concept",
    "conversation_turn": integer,
    "correct_solution": 0 or 1,
    "problem_difficulty": integer_between_1_and_3
  }},
  "action": {{
    "next_action_hint_strength": integer_between_0_and_4,
    "next_action_category": "Focus/Probing/Telling/Generic",
    "next_action_subcategory": "pick_the_exact_one_subcategory_name"
  }}
}}

IMPORTANT:
- Return ONLY the JSON object with no additional explanation
- All numeric fields must be valid integers
"""
    else:
        prompt = f"""
You are analyzing math tutoring sessions to extract features for an AI tutor as you did before.

CONTEXT:
- Question: {question}
- Correct solution: {correct_solution}
- Student profile: {student_profile}
- Current conversation: {current_conversation_state}
- Conversation turn: {conversation_turn}
- Next action text: {action_text}

FORMAT YOUR RESPONSE AS JSON:
{{
  "state": {{
    "misconception_type": "one_of_the_six_types",
    "listen_to_feedback": 0 or 1,
    "problem_progress": integer_between_0_and_100,
    "problem_concept": "specific_math_concept",
    "conversation_turn": integer,
    "correct_solution": 0 or 1,
    "problem_difficulty": integer_between_1_and_3
  }},
  "action": {{
    "next_action_hint_strength": integer_between_0_and_4,
    "next_action_category": "Focus/Probing/Telling/Generic",
    "next_action_subcategory": "pick_the_exact_one_subcategory_name"
  }}
}}

IMPORTANT:
- Return ONLY the JSON object with no additional explanation
- All numeric fields must be valid integers
"""

    # Example payload structure; adjust to your actual usage:
    # payload = {
    #     "model": "gpt-4-0613",
    #     "messages": [
    #         {"role": "user", "content": prompt.strip()}
    #     ],
    #     "max_tokens": 400
    # }

    

    processed += 1

    try:
        print(f"Processing row {i} ...")
        
        response = client.chat(model='gemma3:27b', messages=[
              {
                'role': 'user',
                "content": prompt.strip()
              },
            ])
        
        raw_response = response['message']['content']

        if i == 0:
            print(prompt.strip())
            print(raw_response)
            
        # Store raw response
        state_df.loc[i, "raw_model_response"] = raw_response

        data = extract_json(raw_response)
        if data is not None:
            # Validate & store relevant fields
            # Because we do more thorough checks in validate_schema,
            # we can now simply parse them into your df if you like.

            # Flatten 'state' ...
            state_part = data.get("state", {})
            state_df.loc[i, "misconception_type"] = state_part.get("misconception_type", "")
            state_df.loc[i, "listen_to_feedback"] = state_part.get("listen_to_feedback", 0)
            state_df.loc[i, "problem_progress"] = state_part.get("problem_progress", 0)
            state_df.loc[i, "problem_concept"] = state_part.get("problem_concept", "")
            state_df.loc[i, "conversation_turn"] = state_part.get("conversation_turn", 0)
            state_df.loc[i, "correct_solution"] = state_part.get("correct_solution", 0)
            state_df.loc[i, "problem_difficulty"] = state_part.get("problem_difficulty", 3)  # Default to middle value
            # state_df.loc[i, "reasoning"] = state_part.get("reasoning", "")

            # Flatten 'action' ...
            action_part = data.get("action", {})
            state_df.loc[i, "next_action_hint_strength"] = action_part.get("next_action_hint_strength", 1)
            state_df.loc[i, "next_action_category"] = action_part.get("next_action_category", "")
            state_df.loc[i, "next_action_subcategory"] = action_part.get("next_action_subcategory", "")

            print(f"Successfully extracted JSON for row {i}")
            successful += 1
        else:
            state_df.loc[i, "extraction_failed"] = True
            print(f"Row {i}: Failed to extract valid JSON.")
            failed += 1

        # Save partial progress every 50 rows
        if i % 50 == 0:
            state_df.to_csv("mathdial_gemma3_27b_in_progress.csv", index=False)
            print(f"Progress saved at row {i}.")

    except Exception as e:
        state_df.loc[i, "error_message"] = str(e)
        state_df.loc[i, "extraction_failed"] = True
        print(f"Error processing row {i}: {str(e)}")
        failed += 1

    print("--------------------------------------------------------")

# Final save
state_df.to_csv("mathdial_gemma3_27b_complete.csv", index=False)

print(f"\nProcessing complete:")
print(f"Total rows processed: {processed}")
print(f"Successful extractions: {successful}")
print(f"Failed extractions: {failed}")
if processed > 0:
    print(f"Success rate: {successful/processed*100:.2f}%")
else:
    print("No rows processed.")

Processing row 0 ...
You are analyzing math tutoring sessions to extract features for an AI tutor.

CONTEXT:
- Question: Nancy is filling an aquarium for her fish. She fills it halfway and goes to answer the door. While she's gone, her cat knocks the aquarium over and spills half the water in it. Then Nancy comes back and triples the amount of water in the aquarium. If the aquarium is 4 feet long, 6 feet wide, and 3 feet high, how many cubic feet of water are in the aquarium?
- Correct solution: First calculate the volume of the aquarium by multiplying its length, width and height: 4 ft * 6 ft * 3 ft = 72 cubic ft
Then figure out what proportion of the aquarium is full after the cat knocks it over: 1/2 * 1/2 = 1/4
Then figure out what proportion of the aquarium is full after Nancy refills it: 3 * 1/4 = 3/4
Now multiply the proportion of the aquarium that's full by the aquarium's volume to find out how much water is in it: 72 cubic ft * 3/4 = 54 cubic ft
 54
- Student profile: Steven is

KeyboardInterrupt: 